In [843]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [852]:
# load the data

exercises_raw = pd.read_csv('data/megaGymDataset.csv')
exercies = exercises_raw['Title'].unique()

user_ratings_raw = pd.read_csv('data/user_rating_data.csv')
users = user_ratings_raw['User ID'].unique()

feature_type = exercises_raw['Type'].unique()

num_users = len(users)
num_exercises = len(exercies)
num_feature_type = len(feature_type)
num_recommmendations = 5



In [853]:
import sklearn
from sklearn import preprocessing

df = pd.read_csv('data/user_rating_data.csv')

r = df['Rating'].values.astype(np.float32)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(r.reshape(-1, 1))
df_normalized = pd.DataFrame(x_scaled)
df['Rating'] = df_normalized



In [854]:
import tensorflow as tf
import numpy as np
df = pd.read_csv('data/user_rating_data.csv')

# Define the shape of the matrix
matrix_recos = df.pivot(index='User ID', columns='Exercise', values='Rating').fillna(0)

matrix_recos 

Exercise,Cardio,Olympic Weightlifting,Plyometrics,Powerlifting,Strength,Stretching,Strongman
User ID,,,,,,,
0240de04-1ef3-4d1b-8ab1-09477185b5df,0,0,3,4,1,1,2
025cb182-d876-4574-9f55-036841f7222b,2,0,0,0,1,0,4
0383813b-386b-44c5-bb01-a3b801a0771a,1,2,5,3,5,3,2
05a3d754-f3a6-4013-8a2b-b0723579268f,1,4,0,0,0,0,0
08f05070-db91-41e5-ba59-5e85efd01431,1,3,0,4,0,1,0
...,...,...,...,...,...,...,...
f3682b5f-2037-488d-89a6-ba27fb4fa63b,2,0,0,0,0,0,0
f6b28a51-0891-452a-bf41-b492523030a4,0,0,5,3,0,3,0
f80c8104-12e5-4d81-8dab-140a50ceb1a5,1,1,0,2,3,0,2


In [855]:
df = pd.read_csv('data/megaGymDataset.csv')
df = df.drop_duplicates(subset=['Title', 'Type'], keep='first')

# generate dictionary of exercise and type
exercise_type = {}
for i in df['Title']: 
    if i not in exercise_type.keys():
        exercise_type[i] = df[df['Title'] == i]['Type'].values[0]
    else:
        continue
    

exercise_type

{'Partner plank band row': 'Strength',
 'Banded crunch isometric hold': 'Strength',
 'FYR Banded Plank Jack': 'Strength',
 'Banded crunch': 'Strength',
 'Crunch': 'Strength',
 'Decline band press sit-up': 'Strength',
 'FYR2 Banded Frog Pump': 'Strength',
 'Band low-to-high twist': 'Strength',
 'Barbell roll-out': 'Strength',
 'Barbell Ab Rollout - On Knees': 'Strength',
 'Decline bar press sit-up': 'Strength',
 'Bench barbell roll-out': 'Strength',
 'Barbell Side Bend': 'Strength',
 'Seated bar twist': 'Strength',
 'Single-arm landmine pull and press': 'Strength',
 'Barbell Ab Roll Out - Gethin Variation': 'Strength',
 '30 Barbell Floor Wiper': 'Strength',
 '30 Barbell Roll-Out': 'Strength',
 'Decline plate sit-up': 'Strength',
 'KV Barbell Hip Thrust': 'Strength',
 'Advanced Kettlebell Windmill': 'Strength',
 'Kettlebell Windmill': 'Strength',
 'Kettlebell Pass Between The Legs': 'Strength',
 'Kettlebell 3-point leg extension': 'Strength',
 'Double Kettlebell Swing': 'Strength',
 'Bur

In [856]:
df = pd.read_csv('data/megaGymDataset.csv')
df = df.drop_duplicates(subset=['Title', 'Type'], keep='first')


matrix_feats = df.pivot(index='Title', columns='Type', values='Rating').fillna(0)


for i in range(matrix_feats.shape[0]):
    for j in range(matrix_feats.shape[1]):
        # if dictionary value is in column name, set value to 1
        # else set value to 0
        if exercise_type[matrix_feats.index[i]] in matrix_feats.columns[j]:
            matrix_feats.iloc[i,j] = 1



matrix_feats

Type,Cardio,Olympic Weightlifting,Plyometrics,Powerlifting,Strength,Stretching,Strongman
Title,,,,,,,
1.5-rep push-up,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3/4 sit-up,0.0,0.0,0.0,0.0,1.0,0.0,0.0
30 Arms BFR Close-Grip Push-Up,0.0,0.0,0.0,0.0,1.0,0.0,0.0
30 Arms BFR Dumbbell Kick-Back,0.0,0.0,0.0,0.0,1.0,0.0,0.0
30 Arms BFR High Cable Curl,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
Yates Row,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Yates Row Reverse Grip,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Yoga plex,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [857]:
m1 = matrix_recos.to_numpy()
m2 = matrix_feats.to_numpy().transpose()

In [858]:


# convert to numpy array

matrix_recos = tf.constant(m1, dtype=tf.float32)
matrix_feats = tf.constant(m2, dtype=tf.float32)


user_features = tf.multiply(matrix_recos, matrix_feats).numpy()





ValueError: Dimensions must be equal, but are 100 and 7 for '{{node Mul_4}} = Mul[T=DT_FLOAT](Const_72, Const_73)' with input shapes: [100,7], [7,2909].

In [ ]:
user_features

<tf.Tensor 'truediv_9:0' shape=(100, 2909) dtype=float32>

In [823]:
user_features = user_features/tf.reduce_sum(user_features, axis=1, keepdims=True)


In [799]:
top_user_features = tf.nn.top_k(user_features, num_feature_type) 

In [803]:
for i in range(num_users):
    print('User: ', users[i])
    print('Top 5 recommended exercises: ', top_user_features.eval())



User:  9385c2c1-5e3a-4ec8-b083-fd5bfe9c1e68


AttributeError: 'TopKV2' object has no attribute 'eval'